In [1]:
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):
    model_config = SettingsConfigDict(extra="ignore")

    openai_api_key: str
    database_path: str = "./data"


settings = Settings()

In [2]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

chroma_client = chromadb.PersistentClient(path=settings.database_path)
collection = chroma_client.get_or_create_collection(
    name="docs",
    embedding_function=OpenAIEmbeddingFunction(
        model_name="text-embedding-3-small", api_key=settings.openai_api_key
    ),
)

In [3]:
collection.upsert(
    ids=["1", "2", "3"],
    documents=[
        "LangChain is a framework for building applications with LLMs.",
        "RAG stands for Retrieval-Augmented Generation.",
        "FAISS is Facebook AI Similarity Search for vector indexing.",
    ],
)

results = collection.query(query_texts=["What is Rag?"], n_results=1)
print(results)

{'ids': [['2']], 'embeddings': None, 'documents': [['RAG stands for Retrieval-Augmented Generation.']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None]], 'distances': [[0.6730250716209412]]}
